In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException, TimeoutException

import pandas as pd
from bs4 import BeautifulSoup
import time

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--start-maximized")  # 視窗最大化
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

url = "http://192.192.231.232/bookDetail.do?id=260965&resid=188809503&nowid=3"
ISBN="9789573317241"

In [2]:
driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
wait = WebDriverWait(driver, 30)  # 最長等待時間：30 秒；查找元素的間隔時間：0.5秒（default）

driver.get(url)

In [10]:
tgt = pd.read_html(driver.page_source, encoding="utf-8")[-2]
tgt

,#,條碼號,館藏地/室,索書號,資料類型,目前狀態 / 到期日,附件/備註,預約
0,1,C093472,新竹分館/新竹分館,Book 873.57 8666,中文書/一般,書在館,NaN,不可預約


In [ ]:
tgt.cloumns[]